In [12]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage import io, color, img_as_float
import random
from scipy.optimize import minimize
import json
import os
from abc import ABC, abstractmethod
import numpy as np
from PIL import Image

### Определяем функцию MSE и функцию сравнения изображений

In [9]:
def mse(imageA, imageB):
    """Mean Squared Error"""
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def compare_images(imageA, imageB):
    """
    Сравнение двух изображений: MSE и SSIM.
    """
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB)
    return m, s

### Реализация класса Optimizer с использованием паттерна "Стратегия"

In [10]:
class Optimizer(ABC):
    def __init__(self, method):
        self.method = method

    @abstractmethod
    def optimize(self, objective_function, bounds):
        pass

class MonteCarloOptimizer(Optimizer):
    def __init__(self):
        super().__init__("monte_carlo")

    def optimize(self, objective_function, bounds, max_iterations=1000):
        best_point = None
        best_value = float('inf')
        for _ in range(max_iterations):
            point = [random.uniform(b[0], b[1]) for b in bounds]
            value = objective_function(point)
            if value < best_value:
                best_value = value
                best_point = point
        return best_point, best_value

class GridSearchOptimizer(Optimizer):
    def __init__(self):
        super().__init__("grid_search")

    def optimize(self, objective_function, bounds, steps=10):
        best_point = None
        best_value = float('inf')
        grids = [np.linspace(b[0], b[1], steps) for b in bounds]
        for point in np.array(np.meshgrid(*grids)).T.reshape(-1, len(bounds)):
            value = objective_function(point)
            if value < best_value:
                best_value = value
                best_point = point
        return best_point, best_value


### Реализация класса CalculationManager с использованием паттерна "Одиночка"

In [11]:
class CalculationManager:
    __instance = None

    def __new__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance = super(CalculationManager, cls).__new__(cls)
        return cls.__instance

    def __init__(self):
        if not hasattr(self, 'initialized'):
            self.results = []
            self.initialized = True

    def save_result(self, method_info, best_point, best_accuracy):
        result = {
            "method": method_info,
            "best_point": best_point,
            "best_accuracy": best_accuracy
        }
        self.results.append(result)
        with open("results.json", "w") as f:
            json.dump(self.results, f, indent=4)

    def get_results(self):
        return self.results


###  Функция генерации изображений

In [13]:
# Пути к папкам
blood_cell_dir = 'BCCD_my_Bloode/blood_cell'
background_dir = 'BCCD_my_Bloode/background'

# Получаем списки файлов в папках
blood_cells = [f for f in os.listdir(blood_cell_dir) if f.endswith('.png')]
backgrounds = [f for f in os.listdir(background_dir) if f.endswith('.png')]

def generate_image(x, template_shape):
    """
    Функция генерации изображения на основе параметров x.

    Параметры:
        x: Вектор параметров для управления генерацией изображения.
           Например, x[0] - индекс фона, x[1:] - координаты клеток крови.
        template_shape: Форма шаблона (высота, ширина).

    Возвращает:
        image: Сгенерированное изображение в виде массива NumPy.
    """
    # Определяем размеры шаблона
    height, width = template_shape

    # Выбираем фоновое изображение на основе первого параметра x[0]
    background_index = int(x[0] % len(backgrounds))  # Индекс фона
    background_img = backgrounds[background_index]
    background_path = os.path.join(background_dir, background_img)

    # Загружаем фон и масштабируем его до размера шаблона
    background = cv2.imread(background_path)
    background = cv2.resize(background, (width, height))

    # Создаем новое изображение с фоном
    new_image = background.copy()

    # Количество клеток крови определяется следующим параметром x[1]
    num_cells = max(1, int(x[1] % 30))  # Ограничиваем количество клеток от 1 до 30

    # Генерация позиций клеток крови на основе оставшихся параметров x[2:]
    blood_cell_positions = []
    for i in range(num_cells):
        # Используем параметры x для вычисления позиций клеток
        cell_x = int(x[2 + 2 * i] % width)  # Координата x
        cell_y = int(x[3 + 2 * i] % height)  # Координата y
        blood_cell_positions.append((cell_x, cell_y))

    # Добавляем клетки крови на фон
    for pos in blood_cell_positions:
        # Случайно выбираем клетку крови
        blood_cell_img = random.choice(blood_cells)
        blood_cell_path = os.path.join(blood_cell_dir, blood_cell_img)

        # Открываем изображение клетки крови и конвертируем в RGBA
        blood_cell = Image.open(blood_cell_path).convert("RGBA")
        blood_cell_cv = cv2.cvtColor(np.array(blood_cell), cv2.COLOR_RGBA2BGRA)

        # Создаем маску из альфа-канала
        mask = blood_cell_cv[:, :, 3]
        blood_cell_cv = blood_cell_cv[:, :, :3]

        # Вычисляем центр области для seamlessClone
        center = (pos[0] + blood_cell.width // 2, pos[1] + blood_cell.height // 2)

        # Накладываем клетку крови на фон с использованием seamlessClone
        try:
            new_image = cv2.seamlessClone(
                blood_cell_cv,
                new_image,
                mask,
                center,
                cv2.NORMAL_CLONE
            )
        except:
            # В случае ошибок (например, если область выходит за границы)
            pass

    # Преобразуем изображение в оттенки серого
    new_image_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Масштабируем изображение до диапазона [0, 1]
    new_image_normalized = new_image_gray / 255.0

    return new_image_normalized


### Целевая функция

In [15]:
def objective_function(x, template):
    # Генерируем изображение на основе параметров x
    image = generate_image(x, template.shape)
    # Вычисляем MSE
    return mse(image, template)

###  Обработка сгенерированного изображений из папки

In [16]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            img = io.imread(img_path, as_gray=True)
            images.append(img_as_float(img))
    return images

### Основной блок выполнения

In [5]:
import numpy as np
from PIL import Image
import os

# Функция для загрузки изображений
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                images.append(img)
            except Exception as e:
                print(f"Не удалось загрузить изображение {filename}: {e}")
    return images

# Пример реализации MonteCarloOptimizer
class MonteCarloOptimizer:
    def __init__(self):
        pass

    def optimize(self, objective_function, bounds):
        # Простая реализация случайного поиска
        best_point = [np.random.uniform(low, high) for (low, high) in bounds]
        best_accuracy = objective_function(best_point)
        return best_point, best_accuracy

# Пример реализации CalculationManager
class CalculationManager:
    def __init__(self):
        self.results = []

    def save_result(self, method_info, best_point, best_accuracy):
        self.results.append({
            "method": method_info["method"],
            "bounds": method_info["bounds"],
            "best_point": best_point,
            "best_accuracy": best_accuracy
        })

    def get_results(self):
        return self.results

# Пример целевой функции
def objective_function(x, img):
    # Пример простой целевой функции
    return sum(x)

if __name__ == "__main__":
    # Загрузка изображений
    folder_path = "result_images"
    images = load_images_from_folder(folder_path)

    # Пример шаблона
    template = np.array([[0, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 0, 0],
                         [0, 0, 0, 0]])

    # Определение границ параметров (пример)
    bounds = [(1, 5), (0, 3), (0, 3)]

    # Создание экземпляров классов
    optimizer = MonteCarloOptimizer()
    calculation_manager = CalculationManager()

    # Оптимизация для каждого изображения
    for img in images:
        def objective(x):
            return objective_function(x, img)

        best_point, best_accuracy = optimizer.optimize(objective, bounds)
        calculation_manager.save_result({'method': 'monte_carlo', 'bounds': bounds}, best_point, best_accuracy)

    # Печать результатов
    results = calculation_manager.get_results()
    for result in results:
        print("Метод:", result["method"])
        print("Лучшая точка:", result["best_point"])
        print("Лучшая точность:", result["best_accuracy"])

Метод: monte_carlo
Лучшая точка: [3.59311869482653, 0.7287481515244324, 2.849800511886098]
Лучшая точность: 7.171667358237061
Метод: monte_carlo
Лучшая точка: [4.381597423056983, 1.1190739059900734, 0.702907974129353]
Лучшая точность: 6.20357930317641
Метод: monte_carlo
Лучшая точка: [2.70173471296346, 1.5999265777725986, 1.7825439475660176]
Лучшая точность: 6.084205238302076
Метод: monte_carlo
Лучшая точка: [1.3396687229232302, 0.5447151099007329, 2.791890743539333]
Лучшая точность: 4.676274576363296
Метод: monte_carlo
Лучшая точка: [2.079440619116908, 1.1830640948803695, 2.6660291158879903]
Лучшая точность: 5.928533829885268
Метод: monte_carlo
Лучшая точка: [3.5452499284251586, 0.9149086958205914, 1.4681316242295712]
Лучшая точность: 5.928290248475321
Метод: monte_carlo
Лучшая точка: [1.3750256845538051, 0.37858589866344017, 0.23649525042535902]
Лучшая точность: 1.9901068336426042
Метод: monte_carlo
Лучшая точка: [3.2333355750705426, 1.9320236105431352, 0.34283778892887595]
Лучшая то